In [ ]:

"""
Generate DFT images
"""

import cv2
import os
import torch
import random
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.models as models
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import foolbox as fb
from keras.datasets import cifar10

In [ ]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
def generate_dft(model, data_x, data_y, label, image_type, output_name):
    """
    model: target model to adversarial attack
    data: seem images
    label: label of model
    image_type: type for adversarial DFT
    """
    if image_type == 'test':
        scope = range(9)
    elif image_type == 'val':
        scope = range(9,18)
    elif image_type == 'train':
        scope = range(18,68)
    
    filepath_temp = './images/temp'
    filepath_forder = './images/'+image_type
    filepath ='./images/'+image_type+'/'+label
    if not os.path.exists(filepath_forder):
        os.makedirs(filepath_forder)
    if not os.path.exists(filepath_temp):
        os.makedirs(filepath_temp)
    if not os.path.exists(filepath):
        os.makedirs(filepath)

    bounds = (0,255)
    fmodel = fb.PyTorchModel(model, bounds=bounds)
    for k in scope:
        X_test=torch.from_numpy(data_x[0+32*k:32+32*k]).float().to(device)
        X_test=F.interpolate(X_test, size=(224, 224), mode='bilinear', align_corners=False)
        y_test=[]
        for j in range(32):
            y_test.append(torch.argmax(model(X_test)[j]))
        y_test=torch.tensor(y_test).to(device)
        attack = fb.attacks.FGSM()

        raw, clipped, is_adv = attack(fmodel,X_test,y_test, epsilons=0.03)

        for i in range(32):
            plt.figure(num=None, figsize=(2,2), dpi=150)
            plt.imshow((clipped[i]-X_test[i]).cpu().permute(1,2,0));
            plt.axis('off')
            plt.savefig(filepath_temp+'/per'+str(i+32*k)+'.jpg', dpi=150,bbox_inches='tight', pad_inches=0)
        for i in range(32):
            img_c1=cv2.imread(os.path.join(filepath_temp,'per'+str(i+32*k)+'.jpg'), 0)
            img_c2 = np.fft.fft2(img_c1)
            img_c3 = np.fft.fftshift(img_c2)
            cv2.imwrite(os.path.join(filepath,output_name+str(i+32*k)+'.jpg'),20*np.log(1+np.abs(img_c3)))
        for i in range(32):
            os.remove(filepath_temp+'/per'+str(i+32*k)+'.jpg')

In [ ]:
# Specify the cuda setting if you needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#load dataset
(X_train, y_train), (X_test1, y_test1) = cifar10.load_data()
y_test1=tf.keras.utils.to_categorical(y_test1)
X_test1=X_test1.reshape(10000,3,32,32)
X_test1=X_test1/255
X_test1=X_test1.astype(np.float32)
random.seed(11)
random_numbers = np.arange(10000)
random.shuffle(random_numbers)
X_test1 = X_test1[random_numbers]
y_test1= y_test1[random_numbers]

In [ ]:
#Train dataset for DeepTaster
architectures = ['Resnet18','Vgg16','Densenet161']
datasets = ['Cifar10','MNIST','Tiny']

In [ ]:
#Test dataset for DeepTaster
for dataset in range(3):
    for architecture in architectures:
        test_model=torch.load("./models/test/"+datasets[dataset]+"_"+architecture+".pt",map_location='cpu')
        test_model = test_model.to(device)
        test_model.eval()
        generate_dft(test_model, X_test1,y_test1,  str(dataset), 'test', architecture)


In [ ]:
#validattion dataset for DeepTaster
for architecture in architectures:
    val_model=torch.load("./models/victims/Cifar10_"+architecture+".pt",map_location='cpu')
    val_model = val_model.to(device)
    val_model.eval()
    generate_dft(val_model, X_test1,y_test1, '0', 'val', architecture)

In [ ]:
#Train dataset for DeepTaster    
for architecture in architectures :
    #Load Pretrained model
    train_model=torch.load("./models/victims/Cifar10_"+architecture+".pt",map_location='cpu')
    train_model = train_model.to(device)
    train_model.eval()
    generate_dft(train_model, X_test1,y_test1, '0', 'train', architecture)

In [ ]:
#Test dataset (IMAGENET) for DeepTaster 
test_models = []
test_models.append(models.resnet18(pretrained=True))
test_models.append(models.vgg16(pretrained=True))
test_models.append(models.densenet161(pretrained=True))
for i in range(3):
    test_model = test_models[i]
    test_model = test_model.to(device)
    test_model.eval()
    generate_dft(test_model, X_test1,y_test1,  '4', 'test', architectures[i])